In [1]:
import numpy as np
import pandas as pd

In [62]:
model_name = "preresnet-110"
model_name = "densenet-bc-L190-k40"
# model_name = "resnet-110"

In [63]:
og_df = pd.read_csv(f"/home/ubuntu/expert-predictions/data/{model_name}.csv", header=None)

In [64]:
full_model_logits = pd.DataFrame.to_numpy(og_df)
model_logits = pd.DataFrame.to_numpy(og_df)[:, 11:]
model_logits.shape

(10000, 10)

In [69]:
og_argmax = np.argmax(model_logits, 1)

In [67]:
top_prob = 0.25
adv = 0.05
low_prob = (1 - (top_prob * 3) - adv) / 7
for i in range(len(model_logits)):
    argmax = np.argmax(model_logits[i])
    if argmax < 3:
        full_model_logits[i, :] = np.array([0]*11 + [top_prob] * 3 + [low_prob] * 7)
    elif argmax < 6:
        full_model_logits[i, :] = np.array([0]*11 + [low_prob] * 3 + [top_prob] * 3 + [low_prob] * 4)
    elif argmax < 9:
        full_model_logits[i, :] = np.array([0]*11 + [low_prob] * 6 + [top_prob] * 3 + [low_prob] * 1)
    else:
        full_model_logits[i, :] = np.array([0]*11 + [low_prob] * 7 + [top_prob] * 3)

    full_model_logits[i, 11 + argmax] += adv


In [78]:
assert ((og_argmax - 11) == np.argmax(full_model_logits[:, 11:], 1)).all()

In [72]:
full_model_logits[5], full_model_logits[5].sum()

(array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.02857143, 0.02857143, 0.02857143, 0.02857143,
        0.02857143, 0.02857143, 0.07857143, 0.25      , 0.25      ,
        0.25      ]),
 1.0)

In [73]:
# Generate csv for data
df = pd.DataFrame(full_model_logits) #convert to a dataframe
df.to_csv(f"{model_name}_sabotaged.csv",index=False, header=False) #save to file

#Then, to reload:
df = pd.read_csv(f"{model_name}_sabotaged.csv")